In [ ]:
%run multilabel_preprocessing.ipynb

In [ ]:
%run test.ipynb

In [1]:
import optuna
import torch.optim as optim
import torch
import warnings
import torch.nn as nn
import json

warnings.filterwarnings('ignore')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
def compute_weights_multilabel(y, num_classes=10):
    # Ensure y is a 2D array
    if y.ndim == 1:
        y = y.reshape(-1, 1)
    
    # Count occurrences of each class
    class_counts = np.sum(y, axis=0)
    
    # Compute weights
    weights = np.zeros(num_classes)
    n_samples = len(y)
    
    for i in range(num_classes):
        if class_counts[i] > 0:
            weights[i] = (n_samples / (num_classes * class_counts[i]))
        else:
            weights[i] = 1.0  # or any other strategy for classes with no samples
    
    # Normalize weights
    weights = weights / np.sum(weights) * num_classes
    
    # Create a dictionary mapping class labels to weights
    class_weight_dict = {i: weight for i, weight in enumerate(weights)}
    
    class_weights = torch.FloatTensor(list(class_weight_dict.values())).cuda()

    return class_weights

In [ ]:
class_weights = compute_weights_multilabel(y_train)

In [ ]:
num_classes = y_train.shape[1]
num_classes

In [4]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-6, 1e-1)
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128, 256, 512, 1024])
    epochs = trial.suggest_categorical('epochs', [50, 100, 150, 200, 250])
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
    beta1 = trial.suggest_uniform('beta1', 0.5, 0.9999)
    beta2 = trial.suggest_uniform('beta2', 0.9, 0.9999)
    dropout = trial.suggest_uniform('dropout', 0.0, 0.5)

    input_dim = X_train.shape[1]
    output_dim = num_classes

    model = CNNModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_prob=dropout).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)

    model = training_model(model, optimizer, criterion, train_loader, test_loader, epochs, device)
    accuracy, precision, recall, f1 = test_model(model, test_loader, device)

    return accuracy, precision, recall, f1

In [5]:
study = optuna.load_study(study_name='cnn', storage='sqlite:///multilabelclassification.db')
# study.optimize(objective, n_trials=2000)

In [6]:
trial_number = 100
trial = study.trials[trial_number]

# Extract the parameters
params = trial.params

# Save the parameters as a JSON file
with open(f'trial_{trial_number}_params.json', 'w') as f:
    json.dump(params, f, indent=4)